## Simple Test:
Duty cycle PWM signal on the 0th channel.

In [1]:
import busio
from board import SCL, SDA
from adafruit_pca9685 import PCA9685

In [2]:
i2c_bus = busio.I2C(SCL, SDA)
pca = PCA9685(i2c_bus)
pca.frequency = 60

Opening bus...
Device opened: <_io.FileIO name='/dev/i2c-1' mode='rb+' closefd=True>


In [11]:
# 0xffff: always high; 0x7fff: half high; 0: always low
pca.channels[3].duty_cycle = 0xffff

In [12]:
pca.channels[3].duty_cycle = 0

## Calibration

In [11]:
# SPDX-FileCopyrightText: 2021 ladyada for Adafruit Industries
# SPDX-License-Identifier: MIT

# This advanced example can be used to compute a more precise reference_clock_speed. Use an
# oscilloscope or logic analyzer to measure the signal frequency and type the results into the
# prompts. At the end it'll give you a more precise value around 25 mhz for your reference clock
# speed.


import time

from board import SCL, SDA
import busio

# Import the PCA9685 module.
from adafruit_pca9685 import PCA9685

# Create the I2C bus interface.
i2c_bus = busio.I2C(SCL, SDA)

# Create a simple PCA9685 class instance.
pca = PCA9685(i2c_bus)

# Set the PWM frequency to 100hz.
pca.frequency = 100

input("Press enter when ready to measure default frequency.")

# Set the PWM duty cycle for channel zero to 50%. duty_cycle is 16 bits to match other PWM objects
# but the PCA9685 will only actually give 12 bits of resolution.
print("Running with default calibration")
pca.channels[0].duty_cycle = 0x7FFF
time.sleep(1)
pca.channels[0].duty_cycle = 0

measured_frequency = float(input("Frequency measured: "))
print()

pca.reference_clock_speed = pca.reference_clock_speed * (
    measured_frequency / pca.frequency
)
# Set frequency again so we can get closer. Reading it back will produce the real value.
pca.frequency = 100

input("Press enter when ready to measure coarse calibration frequency.")
pca.channels[0].duty_cycle = 0x7FFF
time.sleep(1)
pca.channels[0].duty_cycle = 0
measured_after_calibration = float(input("Frequency measured: "))
print()

reference_clock_speed = measured_after_calibration * 4096 * pca.prescale_reg

print("Real reference clock speed: {0:.0f}".format(reference_clock_speed))

Opening bus...
Device opened: <_io.FileIO name='/dev/i2c-1' mode='rb+' closefd=True>
Press enter when ready to measure default frequency.
Running with default calibration
Frequency measured: 100

Press enter when ready to measure coarse calibration frequency.
Frequency measured: 100

Real reference clock speed: 24985600


In [5]:
# SPDX-FileCopyrightText: 2021 ladyada for Adafruit Industries
# SPDX-License-Identifier: MIT

import time

from board import SCL, SDA
import busio

# Import the PCA9685 module. Available in the bundle and here:
#   https://github.com/adafruit/Adafruit_CircuitPython_PCA9685
from adafruit_motor import servo
from adafruit_pca9685 import PCA9685

i2c = busio.I2C(SCL, SDA)

# Create a simple PCA9685 class instance.
pca = PCA9685(i2c)
# You can optionally provide a finer tuned reference clock speed to improve the accuracy of the
# timing pulses. This calibration will be specific to each board and its environment. See the
# calibration.py example in the PCA9685 driver.
# pca = PCA9685(i2c, reference_clock_speed=25630710)
pca.frequency = 50

# To get the full range of the servo you will likely need to adjust the min_pulse and max_pulse to
# match the stall points of the servo.
# This is an example for the Sub-micro servo: https://www.adafruit.com/product/2201
# servo7 = servo.Servo(pca.channels[7], min_pulse=580, max_pulse=2350)
# This is an example for the Micro Servo - High Powered, High Torque Metal Gear:
#   https://www.adafruit.com/product/2307
# servo7 = servo.Servo(pca.channels[7], min_pulse=500, max_pulse=2600)
# This is an example for the Standard servo - TowerPro SG-5010 - 5010:
#   https://www.adafruit.com/product/155
# servo7 = servo.Servo(pca.channels[7], min_pulse=400, max_pulse=2400)
# This is an example for the Analog Feedback Servo: https://www.adafruit.com/product/1404
# servo7 = servo.Servo(pca.channels[7], min_pulse=600, max_pulse=2500)
# This is an example for the Micro servo - TowerPro SG-92R: https://www.adafruit.com/product/169
# servo7 = servo.Servo(pca.channels[7], min_pulse=500, max_pulse=2400)

# The pulse range is 750 - 2250 by default. This range typically gives 135 degrees of
# range, but the default is to use 180 degrees. You can specify the expected range if you wish:
# servo7 = servo.Servo(pca.channels[7], actuation_range=135)
servo7 = servo.Servo(pca.channels[15])

# We sleep in the loops to give the servo time to move into position.
for i in range(180):
    servo7.angle = i
    time.sleep(0.03)
for i in range(180):
    servo7.angle = 180 - i
    time.sleep(0.03)

# You can also specify the movement fractionally.
fraction = 0.0
while fraction < 1.0:
    servo7.fraction = fraction
    fraction += 0.01
    time.sleep(0.03)

pca.deinit()

Opening bus...
Device opened: <_io.FileIO name='/dev/i2c-1' mode='rb+' closefd=True>


In [47]:
import busio
import time
from board import SCL, SDA
from adafruit_pca9685 import PCA9685

speed_hex_max = 0xFFFF
clock_wise = "clock_wise"
anti_clock_wise = "anti_clock_wise"

class PCA_Motor:
    """ Basic motor behaviours. """
    
    def __init__(self, channels, frequency=60):
        self.i2c_bus = busio.I2C(SCL, SDA)
        self.pca = PCA9685(i2c_bus)
        self.pca.frequency = frequency
        self.channels = channels
        self.rotation = None
        
    def rotate(self, speed):
        if speed < -1 or speed > 1:
            raise ValueError("invalid speed '%d', should between 0 and 1" %speed)
        
        self.safe_check(speed)
        
        if speed >= 0:
            speed = int(speed*speed_hex_max)
            pca.channels[self.channels["channel_clockwise"]].duty_cycle = speed
            self.rotation = clock_wise
            print("self.rotation", self.rotation)
        elif speed < 0:
            speed = int(abs(speed)*speed_hex_max)
            pca.channels[self.channels["channel_anti_clockwise"]].duty_cycle = speed
            self.rotation = anti_clock_wise
            print("self.rotation", self.rotation)
        
    def stop(self):
        for channel in self.channels:
            pca.channels[self.channels[channel]].duty_cycle = 0
        self.rotation = None
        
    def safe_check(self, speed):
        if (speed > 0 and self.rotation == anti_clock_wise) or (speed < 0 and self.rotation == clock_wise):
            self.stop()
            print("sleeping...")
            time.sleep(1)

In [18]:
channels = {"channel_clockwise":0, "channel_anti_clockwise":1}

In [48]:
motor = PCA_Motor(channels)

Opening bus...
Device opened: <_io.FileIO name='/dev/i2c-1' mode='rb+' closefd=True>


In [51]:
motor.rotate(1)

sleeping...
self.rotation clock_wise


In [53]:
motor.stop()

In [52]:
motor.rotate(-1)

sleeping...
self.rotation anti_clock_wise


In [104]:
class Chassis:
    """Class for basic motor motion and keyboard control behaviour. """
    
    def __init__(self):
        self.left_channels = {"channel_clockwise":0, "channel_anti_clockwise":1}
        self.right_channels = {"channel_clockwise":2, "channel_anti_clockwise":3}
        self.left_motor = PCA_Motor(self.left_channels)
        self.right_motor = PCA_Motor(self.right_channels)
        self.gear_speed_map_f = {1: 0.4, 2: 0.7, 3: 1}
        self.gear_speed_map_b = {-1: -0.25, -2: -0.5}
        
    def forward(self, gear):
        if gear not in self.gear_speed_map_f:
            raise ValueError("invalid gear '%d'" %gear)
        
        speed = self.gear_speed_map_f[gear]
        self.left_motor.rotate(-speed)
        self.right_motor.rotate(speed)
    
    def backward(self, gear):
        if gear not in self.gear_speed_map_b:
            raise ValueError("invalid gear '%d'" %gear)
        
        speed = self.gear_speed_map_b[gear]
        self.left_motor.rotate(-speed)
        self.right_motor.rotate(speed)
        
    def turn_right(self, gear):
        if gear in self.gear_speed_map_f:
            speed_left = self.gear_speed_map_f[gear]
            speed_right = 0.5*self.gear_speed_map_f[gear]
            
            self.left_motor.rotate(-speed_left)
            self.right_motor.rotate(speed_right)
            
        elif gear in gear_speed_map_b:
            speed_left = 0.5*self.gear_speed_map_b[gear]
            speed_right = self.gear_speed_map_b[gear]
            
            self.left_motor.rotate(-speed_left)
            self.right_motor.rotate(speed_right)
        
    def turn_left(self, gear):
        if gear in self.gear_speed_map_f:
            speed_left = 0.5*self.gear_speed_map_f[gear]
            speed_right = self.gear_speed_map_f[gear]
            
            self.left_motor.rotate(-speed_left)
            self.right_motor.rotate(speed_right)
            
        elif gear in self.gear_speed_map_b:
            speed_left = self.gear_speed_map_b[gear]
            speed_right = 0.5*self.gear_speed_map_b[gear]
            
            self.left_motor.rotate(-speed_left)
            self.right_motor.rotate(speed_right)
    
    def spin_clockwise(self):
        speed_left = 1
        speed_right = -1
        self.left_motor.rotate(-speed_left)
        self.right_motor.rotate(speed_right)
    
    def spin_anti_clockwise(self):
        speed_left = -1
        speed_right = 1
        self.left_motor.rotate(-speed_left)
        self.right_motor.rotate(speed_right)
    
    def stop(self):
        self.left_motor.stop()
        self.right_motor.stop()

In [105]:
c = Chassis()

Opening bus...
Device opened: <_io.FileIO name='/dev/i2c-1' mode='rb+' closefd=True>
Opening bus...
Device opened: <_io.FileIO name='/dev/i2c-1' mode='rb+' closefd=True>


In [112]:
c.spin_anti_clockwise()

self.rotation clock_wise
self.rotation clock_wise


In [113]:
c.stop()